In [35]:
import   pandas as pd

In [36]:
df= pd.read_csv('./dataset/synthetic_logs.csv')
df[df.target_label == 'User Action'].log_message.apply(lambda x: print(x))

Account with ID 5351 created by User634.
User User685 logged out.
User User395 logged in.
User User225 logged in.
User User494 logged out.
Account with ID 7813 created by User373.
User User900 logged in.
Account with ID 9827 created by User965.
User User272 logged out.
User User706 logged out.
User User881 logged out.
User User313 logged out.
User User641 logged out.
Account with ID 2520 created by User546.
Account with ID 2300 created by User964.
User User799 logged in.
User User496 logged in.
User User617 logged in.
Account with ID 7505 created by User322.
User User907 logged out.
User User634 logged in.
Account with ID 4810 created by User581.
Account with ID 4995 created by User559.
Account with ID 3889 created by User852.
Account with ID 1478 created by User605.
User User231 logged out.
User User699 logged in.
Account with ID 8884 created by User893.
Account with ID 1314 created by User418.
User User842 logged out.
Account with ID 4725 created by User771.
Account with ID 3189 crea

18      None
27      None
57      None
85      None
88      None
        ... 
2207    None
2263    None
2275    None
2323    None
2353    None
Name: log_message, Length: 144, dtype: object

In [37]:
for col in df.columns:
    if col!='log_message':
        print(df[col].unique())

['2025-06-27 07:20:25' '1/14/2025 23:07' '1/17/2025 1:29' ...
 '2025-08-03 03:07:47' '11/11/2025 11:52' '12/25/2025 13:21']
['ModernCRM' 'AnalyticsEngine' 'ModernHR' 'BillingSystem' 'ThirdPartyAPI'
 'LegacyCRM']
['HTTP Status' 'Critical Error' 'Security Alert' 'Error'
 'System Notification' 'Resource Usage' 'User Action' 'Workflow Error'
 'Deprecation Warning']
['bert' 'regex' 'llm']


In [38]:
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings = model.encode(df['log_message'].tolist())
clustering = DBSCAN(eps=0.2, min_samples=1, metric='cosine').fit(embeddings)
df['cluster'] = clustering.labels_
df.head()

,timestamp,source,log_message,target_label,complexity,cluster
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0


In [39]:
clusters = df.groupby('cluster')['log_message'].apply(list)
sorted_clusters = clusters.sort_values(key=lambda x: x.map(len), ascending=False)

In [40]:
print("Clustered Patterns:")
for cluster_id, messages in sorted_clusters.items():
    if len(messages) > 10:
        print(f"Cluster {cluster_id}:")
        for msg in messages[:5]:
            print(f"  {msg}")

Clustered Patterns:
Cluster 0:
  nova.osapi_compute.wsgi.server [req-b9718cd8-f65e-49cc-8349-6cf7122af137 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2675118
  nova.osapi_compute.wsgi.server [req-4895c258-b2f8-488f-a2a3-4fae63982e48 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" HTTP status code -  200 len: 211 time: 0.0968180
  nova.osapi_compute.wsgi.server [req-ee8bc8ba-9265-4280-9215-dbe000a41209 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" RCODE  200 len: 1874 time: 0.2280791
  nova.osapi_compute.wsgi.server [req-f0bffbc3-5ab0-4916-91c1-0a61dd7d4ec2 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "GET /v2

In [41]:
import re
def classify_with_regex(log_message):
    regex_patterns = {
        r"User User\d+ logged (in|out).": "User Action",
        r"Backup (started|ended) at .*": "System Notification",
        r"Backup completed successfully.": "System Notification",
        r"System updated to version .*": "System Notification",
        r"File .* uploaded successfully by user .*": "System Notification",
        r"Disk cleanup completed successfully.": "System Notification",
        r"System reboot initiated by user .*": "System Notification",
        r"Account with ID .* created by .*": "User Action"
    }
    for pattern, label in regex_patterns.items():
        if re.search(pattern, log_message):
            return label
    return None

In [42]:
classify_with_regex("User User123 logged in.")

'User Action'

In [43]:
# Apply regex classification
df['regex_label'] = df['log_message'].apply(lambda x: classify_with_regex(x))
df[df['regex_label'].notnull()]

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
7,10/11/2025 8:44,ModernHR,File data_6169.csv uploaded successfully by us...,System Notification,regex,4,System Notification
14,1/4/2025 1:43,ThirdPartyAPI,File data_3847.csv uploaded successfully by us...,System Notification,regex,4,System Notification
15,5/1/2025 9:41,ModernCRM,Backup completed successfully.,System Notification,regex,8,System Notification
18,2/22/2025 17:49,ModernCRM,Account with ID 5351 created by User634.,User Action,regex,9,User Action
27,9/24/2025 19:57,ThirdPartyAPI,User User685 logged out.,User Action,regex,11,User Action
...,...,...,...,...,...,...,...
2376,6/27/2025 8:47,ModernCRM,System updated to version 2.0.5.,System Notification,regex,21,System Notification
2381,9/5/2025 6:39,ThirdPartyAPI,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2394,4/3/2025 13:13,ModernHR,Disk cleanup completed successfully.,System Notification,regex,32,System Notification
2395,5/2/2025 14:29,ThirdPartyAPI,Backup ended at 2025-05-06 11:23:16.,System Notification,regex,13,System Notification


In [44]:
df[df['regex_label'].isnull()].head(50)

,timestamp,source,log_message,target_label,complexity,cluster,regex_label
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None
5,2025-10-09 10:30:31,ModernHR,nova.osapi_compute.wsgi.server [req-f0bffbc3-5...,HTTP Status,bert,0,None
6,3/1/2025 19:14,ModernHR,Shard 6 replication task ended in failure,Error,bert,3,None
8,2025-02-12 10:42:29,ThirdPartyAPI,nova.compute.claims [req-a07ac654-8e81-416d-bf...,Resource Usage,bert,5,None
9,2025-03-30 04:01:45,ModernHR,nova.osapi_compute.wsgi.server [req-2bf7cfee-a...,HTTP Status,bert,0,None
10,8/9/2025 18:58,ModernCRM,Email server encountered a sending fault,Error,bert,1,None


In [45]:
df_non_regex = df[df['regex_label'].isnull()].copy()
df_non_regex.shape

(1910, 7)

In [46]:
df_legacy = df_non_regex[df_non_regex.source=="LegacyCRM"]
print(df_legacy.head())
df_non_legacy = df_non_regex[df_non_regex.source!="LegacyCRM"]
print(df_non_legacy.head())
print("Legacy shape: ",df_legacy.shape)
print("Non legacy shape: ",df_non_legacy.shape)

                timestamp     source  \
60    2025-10-06 16:55:23  LegacyCRM   
255   2025-05-03 16:55:35  LegacyCRM   
377   2025-06-24 12:16:29  LegacyCRM   
1325  2025-04-17 07:33:44  LegacyCRM   
1734  2025-04-30 07:47:30  LegacyCRM   

                                            log_message         target_label  \
60    Lead conversion failed for prospect ID 7842 du...       Workflow Error   
255   API endpoint 'getCustomerDetails' is deprecate...  Deprecation Warning   
377   Customer follow-up process for lead ID 5621 fa...       Workflow Error   
1325  Escalation rule execution failed for ticket ID...       Workflow Error   
1734  The 'ExportToCSV' feature is outdated. Please ...  Deprecation Warning   

     complexity  cluster regex_label  
60          llm       24        None  
255         llm       48        None  
377         llm       62        None  
1325        llm      105        None  
1734        llm      118        None  
             timestamp           source  \
0

In [47]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight embedding model
embeddings_filtered = model.encode(df_non_legacy['log_message'].tolist())
X = embeddings_filtered
y = df_non_legacy['target_label'].values


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("--- Logistic Regression Classification Report ---")
report = classification_report(y_test, y_pred)
print(report)

--- Logistic Regression Classification Report ---
                precision    recall  f1-score   support

Critical Error       0.91      1.00      0.95        48
         Error       0.98      0.89      0.93        47
   HTTP Status       1.00      1.00      1.00       304
Resource Usage       1.00      1.00      1.00        49
Security Alert       1.00      0.99      1.00       123

      accuracy                           0.99       571
     macro avg       0.98      0.98      0.98       571
  weighted avg       0.99      0.99      0.99       571



In [49]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

lgb_clf = lgb.LGBMClassifier(objective='multiclass', random_state=42)
lgb_clf.fit(X_train, y_train)

y_pred = lgb_clf.predict(X_test)
print("--- LightGBM Classification Report ---")
report = classification_report(y_test, y_pred)
print(report)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003251 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 97920
[LightGBM] [Info] Number of data points in the train set: 1332, number of used features: 384
[LightGBM] [Info] Start training from score -2.449505
[LightGBM] [Info] Start training from score -2.382252
[LightGBM] [Info] Start training from score -0.631993
[LightGBM] [Info] Start training from score -2.432263
[LightGBM] [Info] Start training from score -1.599725
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

/opt/anaconda3/envs/RCAssist/lib/python3.9/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [50]:
import joblib
joblib.dump(lgb_clf, '../models/log_classifier.joblib')

['../models/log_classifier.joblib']

In [51]:
# ### Unsupervised Severity Classification
# import re
#
# class SeverityClassifier:
#     """
#     An unsupervised classifier to determine the severity of a log message
#     based on predefined keywords.
#     """
#     def __init__(self):
#         self.severity_keywords = {
#             'CRITICAL': ['panic', 'crash', 'critical', 'fatal'],
#             'ERROR': ['error', 'failed', 'failure', 'denied', 'exception'],
#             'WARNING': ['warning', 'deprecated', 'outdated', 'bad login', 'invalid'],
#             'INFO': ['status', 'logged in', 'logged out', 'completed', 'reboot', 'updated', 'uploaded']
#         }
#
#     def classify(self, log_message):
#         """
#         Classifies a single log message into a severity level.
#         The method checks for keywords in a specific order:
#         CRITICAL -> ERROR -> WARNING -> INFO.
#         The first match determines the severity level.
#         """
#         log_lower = log_message.lower()
#
#         # Check for the most severe keywords first
#         if any(re.search(r'\b' + keyword + r'\b', log_lower) for keyword in self.severity_keywords['CRITICAL']):
#             return 'CRITICAL'
#
#         if any(re.search(r'\b' + keyword + r'\b', log_lower) for keyword in self.severity_keywords['ERROR']):
#             return 'ERROR'
#
#         if any(re.search(r'\b' + keyword + r'\b', log_lower) for keyword in self.severity_keywords['WARNING']):
#             return 'WARNING'
#
#         if any(re.search(r'\b' + keyword + r'\b', log_lower) for keyword in self.severity_keywords['INFO']):
#             return 'INFO'
#
#         # Default if no keywords are found
#         return 'UNKNOWN'
#
# # Instantiate the classifier and apply it to the dataframe
# severity_classifier = SeverityClassifier()
# df['predicted_severity'] = df['log_message'].apply(severity_classifier.classify)
#
# # Check the distribution of the predicted severity levels
# print("Distribution of Predicted Severity Levels:")
# print(df['predicted_severity'].value_counts())
#
# # Display the dataframe with the new severity column
# df.head()

Distribution of Predicted Severity Levels:
predicted_severity
UNKNOWN     1200
INFO         958
ERROR        154
WARNING       52
CRITICAL      46
Name: count, dtype: int64


,timestamp,source,log_message,target_label,complexity,cluster,regex_label,predicted_severity
0,2025-06-27 07:20:25,ModernCRM,nova.osapi_compute.wsgi.server [req-b9718cd8-f...,HTTP Status,bert,0,None,INFO
1,1/14/2025 23:07,ModernCRM,Email service experiencing issues with sending,Critical Error,bert,1,None,UNKNOWN
2,1/17/2025 1:29,AnalyticsEngine,Unauthorized access to data was attempted,Security Alert,bert,2,None,UNKNOWN
3,2025-07-12 00:24:16,ModernHR,nova.osapi_compute.wsgi.server [req-4895c258-b...,HTTP Status,bert,0,None,INFO
4,2025-06-02 18:25:23,BillingSystem,nova.osapi_compute.wsgi.server [req-ee8bc8ba-9...,HTTP Status,bert,0,None,UNKNOWN
